<div dir="rtl" style="width:90%; margin:auto; font-family:vazirmatn;">
<ol><li><p><strong>Equality Comparison (مقایسه تساوی):</strong></p><ul><li>این مفهوم به چگونگی تعیین تساوی بین دو شیء اشاره دارد. در .NET، برای اینکه یک نوع داده بتواند به‌درستی به‌عنوان کلید در یک <code>Dictionary</code> یا <code>Hashtable</code> استفاده شود، باید دارای یک پیاده‌سازی از متدهای <code>Equals</code> و <code>GetHashCode</code> باشد. این متدها مسئول تشخیص اینکه آیا دو شیء مساوی هستند و همچنین تولید یک کد هش منحصر به فرد برای هر شیء هستند.</li></ul></li><li><p><strong>Order Comparison (مقایسه ترتیب):</strong></p><ul><li>این مفهوم به چگونگی مقایسه ترتیب بین دو شیء اشاره دارد. در .NET، اگر یک نوع داده پیاده‌سازی از اینترفیس‌های <code>IComparable</code> یا <code>IComparable&lt;T&gt;</code> داشته باشد، می‌تواند به‌عنوان کلید در دیکشنری‌ها یا لیست‌های مرتب استفاده شود. این اینترفیس‌ها برای ترتیب‌دهی اشیاء بر اساس یک ترتیب مشخص (مانند صعودی یا نزولی) استفاده می‌شوند.</li></ul></li></ol>

<h5>Default Behavior (رفتار پیش‌فرض):</h5>
<ul><li>رفتار پیش‌فرض هر نوع داده برای مقایسه و تساوی، معمولاً بازتابی از طبیعی‌ترین رفتار آن نوع است. برای مثال، مقایسه دو رشته ممکن است به صورت پیش‌فرض به حروف بزرگ و کوچک حساس باشد. با این حال، گاهی اوقات رفتار پیش‌فرض همان چیزی نیست که شما نیاز دارید.</li></ul>

<h5>Plug-in Protocols (پروتکل‌های پلاگین):</h5>
<ul><li><p>برای کنترل و تغییر رفتار تساوی و ترتیب، .NET مجموعه‌ای از پروتکل‌های "پلاگین" را معرفی کرده است. این پروتکل‌ها به شما اجازه می‌دهند که رفتار تساوی یا ترتیب‌دهی پیش‌فرض را تغییر دهید یا از نوع داده‌هایی که به‌صورت ذاتی قابل مقایسه یا مساوی نیستند، در مجموعه‌های داده استفاده کنید.</p></li><li><p>این پروتکل‌ها شامل اینترفیس‌های زیر هستند:</p><ul><li><code>IEqualityComparer</code> و <code>IEqualityComparer&lt;T&gt;</code>:<ul><li>این اینترفیس‌ها برای مقایسه تساوی و تولید کدهای هش پلاگین استفاده می‌شوند. اینترفیس‌های <code>IEqualityComparer</code> توسط <code>Hashtable</code> و <code>Dictionary</code> شناسایی می‌شوند.</li></ul></li><li><code>IComparer</code> و <code>IComparer&lt;T&gt;</code>:<ul><li>این اینترفیس‌ها برای مقایسه ترتیب پلاگین استفاده می‌شوند. اینترفیس‌های <code>IComparer</code> توسط دیکشنری‌ها و مجموعه‌های مرتب شناسایی می‌شوند و همچنین در <code>Array.Sort</code> نیز کاربرد دارند.</li></ul></li></ul></li></ul>

<h5>Structural Equatable and Comparable:</h5>
<ul><li>علاوه بر این، اینترفیس‌های <code>IStructuralEquatable</code> و <code>IStructuralComparable</code> نیز وجود دارند که امکان مقایسه‌های ساختاری (structural comparisons) را روی کلاس‌ها و آرایه‌ها فراهم می‌کنند. این به شما اجازه می‌دهد که اشیاء را بر اساس ساختار درونی‌شان مقایسه کنید.</li></ul>
</div>

### `IEqualityComparer` and `EqualityComparer`

Recall the **requirements** of a `hashtable-based` **dictionary**. It needs answers to two questions for any given `key`:
- Is it the `same` as another?
- What is its `integer hashcode`?

In [ ]:
public interface IEqualityComparer<T>
{
    bool Equals (T x, T y);
    int GetHashCode (T obj);
}
public interface IEqualityComparer // Nongeneric version
{
    bool Equals (object x, object y);
    int GetHashCode (object obj);
}

To write a `custom comparer`, you implement **one** or **both** of these ***interfaces***

an ***alternative*** is to **subclass** the ***abstract*** `EqualityComparer` class

In [ ]:
public abstract class EqualityComparer<T> : IEqualityComparer, IEqualityComparer<T>
{
    // متد استاتیک Default که یک EqualityComparer<T> پیش‌فرض را برمی‌گرداند.
    public static EqualityComparer<T> Default { get; } = (EqualityComparer<T>)ComparerHelpers.CreateDefaultEqualityComparer(typeof(T));

    // متد انتزاعی Equals که باید توسط کلاس‌های مشتق‌شده پیاده‌سازی شود.
    public abstract bool Equals(T x, T y);

    // متد انتزاعی GetHashCode که باید توسط کلاس‌های مشتق‌شده پیاده‌سازی شود.
    public abstract int GetHashCode(T obj);

    // پیاده‌سازی رابط IEqualityComparer.Equals
    bool IEqualityComparer.Equals(object x, object y)
    {
        if (x == y) return true;
        if (x == null || y == null) return false;

        if (x is T xT && y is T yT)
            return Equals(xT, yT);
        else
            throw new ArgumentException("Invalid argument type");
    }

    // پیاده‌سازی رابط IEqualityComparer.GetHashCode
    int IEqualityComparer.GetHashCode(object obj)
    {
        if (obj == null) throw new ArgumentNullException(nameof(obj));

        if (obj is T objT)
            return GetHashCode(objT);
        else
            throw new ArgumentException("Invalid argument type");
    }
}


`EqualityComparer` implements both interfaces; your job is simply to ***override*** the `two abstract methods`.

In [ ]:
public class Customer
{
    public string LastName;
    public string FirstName;

    public Customer (string last, string first)
    {
        LastName = last;
        FirstName = first;
    }
}

public class LastFirstEqComparer : EqualityComparer <Customer>
{
    public override bool Equals (Customer x, Customer y)
        => x.LastName == y.LastName && x.FirstName == y.FirstName;

    public override int GetHashCode (Customer obj)
        => (obj.LastName + ";" + obj.FirstName).GetHashCode();
}

Customer c1 = new Customer ("Bloggs", "Joe");
Customer c2 = new Customer ("Bloggs", "Joe");

Console.WriteLine (c1 == c2); // False
Console.WriteLine (c1.Equals (c2)); // False

var d = new Dictionary<Customer, string>();
d [c1] = "Joe";
Console.WriteLine (d.ContainsKey (c2)); //False

//use EqualityComparer
var eqComparer = new LastFirstEqComparer();
var d1 = new Dictionary<Customer, string> (eqComparer);
d1 [c1] = "Joe";
Console.WriteLine (d1.ContainsKey (c2)); // True

##### `EqualityComparer<T>.Default`

you can use `EqualityComparer<T>.Default` as an alternative to the static `object.Equals` method.  
  
***advantage*** is that it **first checks** whether `T` ***implements*** `IEquatable<T>`, and if so, it **calls** that ***implementation*** instead, ***avoiding*** the `boxing` overhead.

In [ ]:
//his is particularly useful in generic methods
static bool Foo<T> (T x, T y)
{
    bool same = EqualityComparer<T>.Default.Equals (x, y);

    //some other logic

    return same;
}

### `IComparer` and `Comparer`

`Comparers` are used to switch in `custom ordering logic` for **sorted dictionaries** and ***collections***.

**Note** :  
`comparer` is ***useless*** to the ***unsorted dictionaries*** such as `Dictionary`
and `Hashtable`—these require an `IEqualityComparer` to **get hashcodes**.  
  
Similarly, an ***equality comparer*** is ***useless*** for `sorted` dictionaries and collections.

In [ ]:
public interface IComparer
{
    int Compare(object x, object y);
}
public interface IComparer <in T>
{
    int Compare(T x, T y);
}

In [ ]:
public abstract class Comparer<T> : IComparer, IComparer<T>
{
    // پراپرتی استاتیک Default که یک Comparer<T> پیش‌فرض را برمی‌گرداند.
    public static Comparer<T> Default { get; } = (Comparer<T>)ComparerHelpers.CreateDefaultComparer(typeof(T));

    // متد انتزاعی Compare که باید توسط کلاس‌های مشتق‌شده پیاده‌سازی شود.
    public abstract int Compare(T x, T y);

    // پیاده‌سازی رابط IComparer.Compare
    int IComparer.Compare(object x, object y)
    {
        if (x == y) return 0;
        if (x == null) return -1;
        if (y == null) return 1;

        if (x is T xT && y is T yT)
            return Compare(xT, yT);
        else
            throw new ArgumentException("Invalid argument type");
    }
}


In [1]:
class Wish
{
    public string Name;
    public int Priority;

    public Wish (string name, int priority)
    {
        Name = name;
        Priority = priority;
    }
}

class PriorityComparer : Comparer<Wish>
{
    public override int Compare (Wish x, Wish y)
    {
        if (object.Equals (x, y)) return 0; // Optimization
        if (x == null) return -1;
        if (y == null) return 1;
        return x.Priority.CompareTo (y.Priority);
    }
}

var wishList = new List<Wish>();
wishList.Add (new Wish ("Peace", 2));
wishList.Add (new Wish ("Wealth", 3));
wishList.Add (new Wish ("Love", 2));
wishList.Add (new Wish ("3 more wishes", 1));

wishList.Sort (new PriorityComparer());
foreach (Wish w in wishList) Console.Write (w.Name + " | ");

3 more wishes | Peace | Love | Wealth | 

### StringComparer

`StringComparer` is a ***predefined plug-in*** class for `equating` and `comparing` ***strings***

`StringComparer` ***implements both*** `IEqualityComparer` and `IComparer` (and their generic versions)

Because ***StringComparer*** is `abstract`, you ***obtain instances*** via its `static properties`.

In [ ]:
public static StringComparer CurrentCulture { get; }
public static StringComparer CurrentCultureIgnoreCase { get; }
public static StringComparer InvariantCulture { get; }
public static StringComparer InvariantCultureIgnoreCase { get; }
public static StringComparer Ordinal { get; }
public static StringComparer OrdinalIgnoreCase { get; }
public static StringComparer Create (CultureInfo culture,
                                        bool ignoreCase);

In [ ]:
//ordinal case-insensitive dictionary is created such that
//dict["Joe"] and dict["JOE"] mean the same thing:

var dict = new Dictionary<string, int> (StringComparer.OrdinalIgnoreCase);

In [ ]:
//an array of names is sorted, using Australian English:
string[] names = { "Tom", "HARRY", "sheila" };
CultureInfo ci = new CultureInfo ("en-AU");
Array.Sort<string> (names, StringComparer.Create (ci, false));

##### `IStructuralEquatable` and `IStructuralComparable`

<div dir="rtl" style="width:90%; margin:auto; font-family:vazirmatn;">
<ol><li><p><strong>مقایسه ساختاری در Structها:</strong></p><ul><li>در .NET، ساختارها (structs) به‌صورت پیش‌فرض از مقایسه‌ی ساختاری پشتیبانی می‌کنند، یعنی دو شیء از نوع struct برابر در نظر گرفته می‌شوند اگر تمامی فیلدهای آن‌ها برابر باشند.</li></ul></li><li><p><strong>نیاز به مقایسه ساختاری در انواع دیگر:</strong></p><ul><li>گاهی اوقات، نیاز است که از مقایسه‌ی ساختاری در انواع مرکب دیگری مانند آرایه‌ها نیز استفاده شود. اینجا است که اینترفیس‌های <code>IStructuralEquatable</code> و <code>IStructuralComparable</code> وارد عمل می‌شوند.</li></ul></li></ol>

<h5>اینترفیس‌ها:</h5>
<ol><li><strong>IStructuralEquatable:</strong><ul><li>این اینترفیس برای انجام مقایسه‌ی تساوی (equality comparison) در اشیاء مرکب به کار می‌رود. این اینترفیس دو متد دارد:<ul><li><code>Equals</code>: که دو شیء را بر اساس مقایسه ساختاری و با استفاده از یک <code>IEqualityComparer</code> مقایسه می‌کند.</li><li><code>GetHashCode</code>: که یک کد هش برای شیء تولید می‌کند، باز هم با استفاده از یک <code>IEqualityComparer</code>.</li></ul></li></ul></li><li><strong>IStructuralComparable:</strong><ul><li>این اینترفیس برای انجام مقایسه‌ی ترتیب (ordering comparison) در اشیاء مرکب استفاده می‌شود. متد اصلی آن <code>CompareTo</code> است که دو شیء را مقایسه کرده و ترتیب آن‌ها را بر اساس یک <code>IComparer</code> مشخص می‌کند.</li></ul></li></ol>
</div>

In [ ]:
public interface IStructuralEquatable
{
    bool Equals (object other, IEqualityComparer comparer);
    int GetHashCode (IEqualityComparer comparer);
}
public interface IStructuralComparable
{
    int CompareTo (object other, IComparer comparer);
}

In [ ]:
int[] a1 = { 1, 2, 3 };
int[] a2 = { 1, 2, 3 };
IStructuralEquatable se1 = a1; // کلاس Array اینترفیس IStructuralEquatable را پیاده سازی کرده
Console.Write (a1.Equals (a2)); // False
Console.Write (se1.Equals (a2, EqualityComparer<int>.Default)); // True

In [ ]:
string[] a1 = "the quick brown fox".Split();
string[] a2 = "THE QUICK BROWN FOX".Split();
IStructuralEquatable se1 = a1;
bool isTrue = se1.Equals (a2, StringComparer.InvariantCultureIgnoreCase);